In [1]:
import muon as mu 
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xarray as xr
import scanpy as sc
import anndata as ad
from pathlib import Path
import cnmf

/home/users/ymo/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cnmf_obj = cnmf.cNMF(output_dir="/oak/stanford/groups/engreitz/Users/ymo/NMF_re-inplementing/Results/cNMF_100_07102024", name="consensus_NMF")
hundred = cnmf_obj.combine_nmf(100)

Combining factorizations for k=100.


In [3]:
df = pd.read_csv('filename.txt', sep='\t', index_col=0)
/oak/stanford/groups/engreitz/Users/ymo/NMF_re-inplementing/Results/cNMF_100_07102024/consensus_NMF/cnmf_tmp/consensus_NMF.gene_spectra_score.k_100.dt_0_2.df.npz

,ENSG00000001617,ENSG00000001626,ENSG00000001630,ENSG00000002746,ENSG00000003096,ENSG00000003400,ENSG00000003402,ENSG00000003436,ENSG00000003987,ENSG00000003989,...,ENSG00000285218,ENSG00000285219,ENSG00000285245,ENSG00000285280,ENSG00000285373,ENSG00000285407,ENSG00000285424,ENSG00000285454,ENSG00000285467,ENSG00000285471
iter0_topic1,0.008255,0.000000,0.000000,0.000496,0.033972,0.004404,0.000000,0.000000,0.028459,0.043130,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
iter0_topic2,0.000000,0.000000,0.000000,0.001221,0.000000,0.000000,0.000000,0.000000,0.000811,0.013229,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000594,0.000000
iter0_topic3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017186,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.014276,0.0,0.000000,0.000000,0.000000
iter0_topic4,0.000000,0.000000,0.000000,0.000000,0.166423,0.000000,0.000000,0.000000,0.211343,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.077786,0.000000,0.000000
iter0_topic5,0.000000,0.000000,0.009436,0.000000,0.000000,0.000000,0.118582,0.094543,0.000000,0.008025,...,0.00000,0.000000,0.019653,0.011036,0.042885,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iter90_topic96,0.000000,0.000000,0.000000,0.005085,0.000000,0.000000,0.163193,0.000000,0.155222,0.000000,...,0.00000,0.728083,0.000000,0.056450,0.000000,0.000000,0.0,0.000000,0.000000,0.094620
iter90_topic97,0.000000,0.000000,0.201160,0.000000,0.010607,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.19924,0.127060,0.000000,0.000000,0.000000,0.000000,0.0,0.071256,0.000000,0.055389
iter90_topic98,0.000000,0.000000,0.014416,0.000000,0.000000,0.000000,0.001866,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.040251,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.031006
iter90_topic99,0.000000,0.000000,0.000000,0.000000,0.000000,0.000973,0.000000,0.000000,0.017053,0.022262,...,0.00000,0.000000,0.000000,0.001907,0.000000,0.000000,0.0,0.002134,0.000000,0.000000
